<a href="https://colab.research.google.com/github/saguileran/MD-SCPI/blob/main/NoteBooks/pyPARSER_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Conda Colab**

In [1]:
#It will restart the kernel (session), don't worry.
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:32
🔁 Restarting kernel...


install dependecies

In [1]:
%%capture
import sys
!pip -q install py3Dmol
!pip install git+https://github.com/pablo-arantes/biopandas
!mamba install openmmforcefields -c conda-forge -y
!git clone https://github.com/pablo-arantes/ProLIF.git
prolif1 = "cd /content/ProLIF"
prolif2 = "sed -i 's/mdanalysis.*/mdanalysis==2.0.0/' setup.cfg"
prolif3 = "pip install ."

original_stdout = sys.stdout # Save a reference to the original standard output

with open('prolif.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(prolif1)
    print(prolif2)
    print(prolif3)
    sys.stdout = original_stdout # Reset the standard output to its original value

!chmod 700 prolif.sh 2>&1 1>/dev/null
!bash prolif.sh >/dev/null 2>&1
!mamba install -c conda-forge openbabel -y
!mamba install -c conda-forge ambertools -y
!mamba install -c conda-forge parmed -y
!pip install --upgrade MDAnalysis

#load dependencies
from openmm import app, unit
from openmm.app import HBonds, NoCutoff, PDBFile
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.utils import get_data_file_path
import parmed as pmd
from biopandas.pdb import PandasPdb
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import os
import urllib.request  
import numpy as np
import MDAnalysis as mda
import py3Dmol
import pytraj as pt
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from pytraj import matrix
from matplotlib import colors
from IPython.display import set_matplotlib_formats
!wget  https://raw.githubusercontent.com/openforcefield/openff-forcefields/master/openforcefields/offxml/openff_unconstrained-2.0.0.offxml

In [2]:
# Import Google Drive
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [16]:
%%capture
import rdkit
import mdtraj as md
from rdkit import Chem
from rdkit.Chem import AllChem,Draw
from rdkit.Chem.Draw import IPythonConsole
from openbabel import pybel

Protein_PDB_file_name = 'MR.pdb'
Ligand_PDB_file_name = 'AS4.pdb'

Ligand_Protein_PDB_file_name = 'MR-AS4_leap.pdb'
Ligand_Protein_prmtop_file_name = 'MR-AS4.prmtop'
Ligand_Protein_inpcrd_file_name = 'MR-AS4.inpcrd'
Ligand_Protein_nc_file_name = 'prod.nc'


ligand_name = Ligand_PDB_file_name
workDir = '/content/drive/MyDrive/MolecularDynamics_2023/USP/pyLiBELa/' 
file_name = os.path.join(workDir, str(Protein_PDB_file_name)) 

nc_name = os.path.join(workDir, str(Ligand_Protein_nc_file_name)) 
param_name = os.path.join(workDir, str(Ligand_Protein_prmtop_file_name)) 


ligand_pdb = os.path.join(workDir, str(ligand_name))
ligand_pdb2 = os.path.join(workDir, "ligand_H.pdb")
#initial_pdb = os.path.join(workDir, "starting0.pdb")
#starting = os.path.join(workDir, "starting1.pdb")
#starting2 = os.path.join(workDir, "starting2.pdb")
#starting_end = os.path.join(workDir, "starting_end.pdb")

In [ ]:
import pytraj as pt
traj = pt.iterload(nc_name, param_name)
pt.rmsd(traj, mask='@CA', ref=0)
pt.dssp(traj, mask=':2-16')
pt.pca(traj, mask='!@H=', n_vecs=2)

# MD Analysis


Although visualizing your trajectory can be quite useful, sometimes you also want more quantitative data.

Analyses of MD trajectories vary a lot and we do not intend to cover it all here. However, one can make use of MDanalysis or PyTraj to easily analyze simulations. 

Below, you can find a few examples of code snippets that can help you to shed some light on your simulation behavior.

In [15]:
#@title **MM-PBSA method to calculate the binding free energy**
#@markdown **Important:** We will now calculate the interaction energy and solvation free energy for the complex, receptor and ligand and average the results to obtain an estimate of the binding free energy. Please note that we will not perform a calculation of the entropy contribution to binding and so strictly speaking our result will not be a true free energy but could be used to compare against similar systems. We will carry out the binding energy calculation using both the MM-GBSA method and the MM-PBSA method for comparison.

#@markdown Select the GB/SA input parameters,  the "OBC" models (igb=2 and 5) are newer, but appear to give significant improvements and are recommended for most projects (For more information check the Section 4.1 of the [Amber Manual](https://ambermd.org/doc12/Amber20.pdf)):
igb = "2" #@param ["1", "2", "5", "7", "8"]

if igb == "1":
  mbondi = 'mbondi'
elif igb == "2" or igb == "5":
  mbondi = 'mbondi2'
elif igb == "7":
  mbondi = 'bondi'
elif igb == "8":
  mbondi = 'mbondi3'
else:
  pass

Salt_concentration = '0.15' #@param {type:"string"}
fold_MMPBSA = "MMPBSA_igb_" + igb 
#@markdown **Provide output file names below:** 
Output_name = 'FINAL_RESULTS_MMPBSA' #@param {type:"string"}

final_mmpbsa = os.path.join(workDir, Output_name)

if number_frames_analysis > 10:
  stride = number_frames_analysis/10
else:
  stride = 1

f = open("mmpbsa.in", "w")
f.write("""&general """  + "\n"
"""  endframe=""" + str(int(number_frames_analysis)) + """,  interval=""" + str(int(stride)) + """, strip_mask=:WAT:Na+:Cl-:Mg+:K+, """ + "\n"
"""/ """ + "\n"
"""&gb """ + "\n"
""" igb=""" + str(igb) +  """, saltcon=""" + str(Salt_concentration) +  """, """ + "\n"
"""/ """ + "\n"
"""&pb """ + "\n"
""" istrng=""" + str(Salt_concentration) +  """, inp=2, radiopt=0, prbrad=1.4, """ + "\n"
"""/""")
f.close()

amberhome = "source /usr/local/amber.sh"
ante_MMPBSA = "ante-MMPBSA.py  -p " + str(pdb_ref) + " -c com.prmtop -r rec.prmtop -l ligand.prmtop -s :WAT:Na+:Cl-:Mg+:K+ -n :LIG --radii " + str(mbondi) 
MMPBSA = "MMPBSA.py -O -i mmpbsa.in -o " + str(final_mmpbsa) +  ".dat -sp " + str(pdb_ref) + " -cp com.prmtop -rp rec.prmtop -lp ligand.prmtop -y "  + str(traj)
mkdir = "mkdir " + os.path.join(workDir, fold_MMPBSA)
mv = "mv _MMPBSA* com.prmtop rec.prmtop ligand.prmtop reference.frc mmpbsa.in " + os.path.join(workDir, fold_MMPBSA)

original_stdout = sys.stdout # Save a reference to the original standard output

with open('run_MMPBSA.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(amberhome)
    print(ante_MMPBSA)
    print(MMPBSA)
    print(mkdir)
    print(mv)
    sys.stdout = original_stdout # Reset the standard output to its original value

!chmod 700 run_MMPBSA.sh 2>&1 1>/dev/null
!bash run_MMPBSA.sh 2>&1 1>/dev/null

f_mmpbsa = open(final_mmpbsa + '.dat', 'r')
file_contents = f_mmpbsa.read()
print(file_contents)
f_mmpbsa.close()

<module 'pytraj' from '/usr/local/lib/python3.8/site-packages/pytraj/__init__.py'>

In [ ]:
#@title **Interaction Energy**
#@markdown **Important:** To quantify the strength of the interaction between the ligand and the protein, we will compute the nonbonded interaction energy between these two species.  It is important to note that this quantity is NOT a free energy or a binding energy. 

#@markdown **Provide output file names below:** 
Output_name = 'Interaction_energy' #@param {type:"string"}

pt_topology = traj_load.top
restraint_array = pt.select_atoms('!(:WAT) & !(:Na+) & !(:Cl-) & !(:Mg+) & !(:K+) & !(:LIG)', pt_topology)
first_atom = restraint_array[0]
last_atom = restraint_array[-1]
mask = "LIE :LIG @" + str(first_atom+1) + "-" + str(last_atom+1)

lie = pt.analysis.energy_analysis.lie(traj_load, mask=mask, options='cutvdw 12.0 cutelec 12.0 diel 2.0', dtype='dict')

lie_elec = lie['LIE[EELEC]']
lie_vdw = lie['LIE[EVDW]']
lie_total = lie_elec + lie_vdw
lie_total_mean = mean(lie_total)
lie_total_stdev = stdev(lie_total)
print("Interaction Energy Average = " + str("{:.2f}".format(lie_total_mean)) + " \u00B1 " + str("{:.2f}".format(lie_total_stdev)) + " kcal/mol")

time = len(lie_total)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

# Plotting:
ax = plt.plot(time_array, lie_total, alpha=0.6, color = 'blue', linewidth = 1.5, label= "Total Energy")
ax = plt.plot(time_array, lie_elec, alpha=0.6, color = 'green', linewidth = 1.5, label= "Electrostatic Energy")
ax = plt.plot(time_array, lie_vdw, alpha=0.6, color = 'red', linewidth = 1.5, label= "van der Waals Energy")

plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel('Interaction Energy \n (kcal/mol)', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

lie_eelec = pd.DataFrame(lie['LIE[EELEC]'])
lie_eelec.to_csv(os.path.join(workDir, Output_name + "_eelec.csv"))
lie_evdw = pd.DataFrame(lie['LIE[EVDW]'])
lie_evdw.to_csv(os.path.join(workDir, Output_name + "_evdw.csv"))

# **LiBELa**

In [9]:
!pip install git+https://github.com/pyparsing/pyparsing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyparsing/pyparsing to /tmp/pip-req-build-txzhrqic
  Running command git clone --filter=blob:none --quiet https://github.com/pyparsing/pyparsing /tmp/pip-req-build-txzhrqic
  Resolved https://github.com/pyparsing/pyparsing to commit 8e7ed8f2704f8d64bd79c1308c19d2b6fe0f781c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyparsing: filename=pyparsing-3.0.10-py3-none-any.whl size=102246 sha256=771b997a858f198a51aeb75e233e43de35aa1c5a980a05be49af83c93208bbec
  Stored in directory: /tmp/pip-ephem-wheel-cache-76dyzhjz/wheels/0d/d3/ff/5528e199378b7ee7a083f394f86303a6f07ff26b25ec1a19a6
Successfully built pyparsing
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled py

In [4]:
! apt-get install libboost-python1.71-dev openbabel python-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
libboost-python1.71-dev is already the newest version (1.71.0-6ubuntu6).
libboost-python1.71-dev set to manually installed.
The following additional packages will be installed:
  libopenbabel6 libpython2-dev libpython2-stdlib libpython2.7 libpython2.7-dev
  libschroedinger-maeparser1 python-is-python2 python2 python2-dev
  python2-minimal python2.7-dev
Suggested packages:
  python2-doc python-tk
The following NEW packages will be installed:
  libopenbabel6 libpython2-dev libpython2-stdlib libpython2.7 libpython2.7-dev
  libschroedinger-maeparser1 openbabel python-dev-is-python2 python-is-python2
  python2 python2-dev python2-minimal python2.7-dev
0 upgraded, 13 newly installed, 0 to remove and 22 not upgraded.
Need to get 7,654 kB of archives.
After this operation, 35.9 MB of additional disk space will be used.
Get:1 http://arch

In [ ]:
! rm -f pyPARSER*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/src/pyPARSER.cpp
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/src/pyPARSER.h
!g++ -Wall -Wextra -fPIC -shared -I/usr/include/python3.8 pyPARSER.cpp -o pyPARSER.so -L/usr/lib/x86_64-linux-gnu -lboost_python38

--2023-02-28 15:32:47--  https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/src/pyPARSER.cpp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22346 (22K) [text/plain]
Saving to: ‘pyPARSER.cpp’

pyPARSER.cpp        100%[===================>]  21.82K  --.-KB/s    in 0.002s  

2023-02-28 15:32:47 (12.5 MB/s) - ‘pyPARSER.cpp’ saved [22346/22346]

--2023-02-28 15:32:47--  https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/src/pyPARSER.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12042 (12K) [text

In [7]:
!ls

condacolab_install.log	openff_unconstrained-2.0.0.offxml  prolif.sh
drive			ProLIF				   sample_data
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyPARSER: filename=pyparser-1.0-py3-none-any.whl size=4921 sha256=4d025e9b622bb7c5d724d4b5fe04b365feb60c4c15e8e242b6fddeb909d197ba
  Stored in directory: /root/.cache/pip/wheels/d5/c1/97/c8d64328b6d76c0393314e67e06e366a75b7ffdf0406d2afc3
  Created wheel for parse: filename=parse-1.6.5-py3-none-any.whl size=18155 sha256=fa1e76cde17d567b61e237a229ca77289feacc114c9819c2b6433027c36b29a0
  Stored in directory: /root/.cache/pip/wheels/ec/be/db/6c913790b51ae090743152e15158ee7effdaefea9a940330e6
Successfully built pyPARSER parse


In [11]:
#from pyPARSER import *
from pyparsing import *

Input = PARSER()

print(Input.temp)

NameError: ignored

In [13]:
pyparsing.PARSER

NameError: ignored